# ML for Gas Adsorption

## -1. Only if you run this notebook on Colab

If you use this notebook on Colab, please uncomment the lines below (remove the `#`) and execute the cell.

In [1]:
#import sys
#!{sys.executable} -m pip install -U pandas-profiling[notebook]
#!jupyter nbextension enable --py widgetsnbextension
#!pip install --upgrade pandas sklearn holoviews bokeh plotly matplotlib
#!wget https://raw.githubusercontent.com/kjappelbaum/ml_molsim/2022/descriptornames.py
#!mkdir data
#!cd data && wget https://github.com/kjappelbaum/ml_molsim/raw/2022/data/data.csv
#!cd data && wget https://github.com/kjappelbaum/ml_molsim/raw/2022/data/features.csv
# import os, holoviews as hv
# os.environ['HV_DOC_HTML'] = 'true'

## Import packages we will need

In [2]:
# basics
import os
import numpy as np
import pprint as pp

# pandas is used to read/process data
import pandas as pd

# machine learning dependencies
# scaling of data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
# train/test split
from sklearn.model_selection import train_test_split
# model selection
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# the KRR model
from sklearn.kernel_ridge import KernelRidge
# linear model
from sklearn.linear_model import LinearRegression
# pipeline to streamline modeling pipelines
from sklearn.pipeline import Pipeline
# principal component analysis
from sklearn.decomposition import PCA
# polynomial kernel
from sklearn.metrics.pairwise import polynomial_kernel
# Dummy model as baseline
from sklearn.dummy import DummyClassifier, DummyRegressor
# Variance Threshold for feature selection
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
# metrics to measure model performance
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             mean_absolute_error, mean_squared_error, max_error)

# save/load models
import joblib

# For the permutation importance implementation
from joblib import Parallel
from joblib import delayed
from sklearn.metrics import check_scoring
from sklearn.utils import Bunch
from sklearn.utils import check_random_state
from sklearn.utils import check_array

# plotting
import matplotlib.pyplot as plt
%matplotlib inline
from pymatviz.parity import hist_density

RANDOM_SEED = 4242424242
DATA_DIR = 'data'
DATA_FILE = os.path.join(DATA_DIR, 'data.csv')

np.random.seed(RANDOM_SEED)


In [3]:

other_descriptors = ["CellV [A^3]"]

geometric_descriptors = [
    "Di",
    "Df",
    "Dif",
    "density [g/cm^3]",
    "total_SA_volumetric",
    "total_SA_gravimetric",
    "total_POV_volumetric",
    "total_POV_gravimetric",
]

linker_descriptors = [
    "f-lig-chi-0",
    "f-lig-chi-1",
    "f-lig-chi-2",
    "f-lig-chi-3",
    "f-lig-Z-0",
    "f-lig-Z-1",
    "f-lig-Z-2",
    "f-lig-Z-3",
    "f-lig-I-0",
    "f-lig-I-1",
    "f-lig-I-2",
    "f-lig-I-3",
    "f-lig-T-0",
    "f-lig-T-1",
    "f-lig-T-2",
    "f-lig-T-3",
    "f-lig-S-0",
    "f-lig-S-1",
    "f-lig-S-2",
    "f-lig-S-3",
    "lc-chi-0-all",
    "lc-chi-1-all",
    "lc-chi-2-all",
    "lc-chi-3-all",
    "lc-Z-0-all",
    "lc-Z-1-all",
    "lc-Z-2-all",
    "lc-Z-3-all",
    "lc-I-0-all",
    "lc-I-1-all",
    "lc-I-2-all",
    "lc-I-3-all",
    "lc-T-0-all",
    "lc-T-1-all",
    "lc-T-2-all",
    "lc-T-3-all",
    "lc-S-0-all",
    "lc-S-1-all",
    "lc-S-2-all",
    "lc-S-3-all",
    "lc-alpha-0-all",
    "lc-alpha-1-all",
    "lc-alpha-2-all",
    "lc-alpha-3-all",
    "D_lc-chi-0-all",
    "D_lc-chi-1-all",
    "D_lc-chi-2-all",
    "D_lc-chi-3-all",
    "D_lc-Z-0-all",
    "D_lc-Z-1-all",
    "D_lc-Z-2-all",
    "D_lc-Z-3-all",
    "D_lc-I-0-all",
    "D_lc-I-1-all",
    "D_lc-I-2-all",
    "D_lc-I-3-all",
    "D_lc-T-0-all",
    "D_lc-T-1-all",
    "D_lc-T-2-all",
    "D_lc-T-3-all",
    "D_lc-S-0-all",
    "D_lc-S-1-all",
    "D_lc-S-2-all",
    "D_lc-S-3-all",
    "D_lc-alpha-0-all",
    "D_lc-alpha-1-all",
    "D_lc-alpha-2-all",
    "D_lc-alpha-3-all",
]

metalcenter_descriptors = [
    "mc_CRY-chi-0-all",
    "mc_CRY-chi-1-all",
    "mc_CRY-chi-2-all",
    "mc_CRY-chi-3-all",
    "mc_CRY-Z-0-all",
    "mc_CRY-Z-1-all",
    "mc_CRY-Z-2-all",
    "mc_CRY-Z-3-all",
    "mc_CRY-I-0-all",
    "mc_CRY-I-1-all",
    "mc_CRY-I-2-all",
    "mc_CRY-I-3-all",
    "mc_CRY-T-0-all",
    "mc_CRY-T-1-all",
    "mc_CRY-T-2-all",
    "mc_CRY-T-3-all",
    "mc_CRY-S-0-all",
    "mc_CRY-S-1-all",
    "mc_CRY-S-2-all",
    "mc_CRY-S-3-all",
    "D_mc_CRY-chi-0-all",
    "D_mc_CRY-chi-1-all",
    "D_mc_CRY-chi-2-all",
    "D_mc_CRY-chi-3-all",
    "D_mc_CRY-Z-0-all",
    "D_mc_CRY-Z-1-all",
    "D_mc_CRY-Z-2-all",
    "D_mc_CRY-Z-3-all",
    "D_mc_CRY-I-0-all",
    "D_mc_CRY-I-1-all",
    "D_mc_CRY-I-2-all",
    "D_mc_CRY-I-3-all",
    "D_mc_CRY-T-0-all",
    "D_mc_CRY-T-1-all",
    "D_mc_CRY-T-2-all",
    "D_mc_CRY-T-3-all",
    "D_mc_CRY-S-0-all",
    "D_mc_CRY-S-1-all",
    "D_mc_CRY-S-2-all",
    "D_mc_CRY-S-3-all",
]

functionalgroup_descriptors = [
    "func-chi-0-all",
    "func-chi-1-all",
    "func-chi-2-all",
    "func-chi-3-all",
    "func-Z-0-all",
    "func-Z-1-all",
    "func-Z-2-all",
    "func-Z-3-all",
    "func-I-0-all",
    "func-I-1-all",
    "func-I-2-all",
    "func-I-3-all",
    "func-T-0-all",
    "func-T-1-all",
    "func-T-2-all",
    "func-T-3-all",
    "func-S-0-all",
    "func-S-1-all",
    "func-S-2-all",
    "func-S-3-all",
    "func-alpha-0-all",
    "func-alpha-1-all",
    "func-alpha-2-all",
    "func-alpha-3-all",
    "D_func-chi-0-all",
    "D_func-chi-1-all",
    "D_func-chi-2-all",
    "D_func-chi-3-all",
    "D_func-Z-0-all",
    "D_func-Z-1-all",
    "D_func-Z-2-all",
    "D_func-Z-3-all",
    "D_func-I-0-all",
    "D_func-I-1-all",
    "D_func-I-2-all",
    "D_func-I-3-all",
    "D_func-T-0-all",
    "D_func-T-1-all",
    "D_func-T-2-all",
    "D_func-T-3-all",
    "D_func-S-0-all",
    "D_func-S-1-all",
    "D_func-S-2-all",
    "D_func-S-3-all",
    "D_func-alpha-0-all",
    "D_func-alpha-1-all",
    "D_func-alpha-2-all",
    "D_func-alpha-3-all",
]


summed_linker_descriptors = [
    "sum-f-lig-chi-0",
    "sum-f-lig-chi-1",
    "sum-f-lig-chi-2",
    "sum-f-lig-chi-3",
    "sum-f-lig-Z-0",
    "sum-f-lig-Z-1",
    "sum-f-lig-Z-2",
    "sum-f-lig-Z-3",
    "sum-f-lig-I-0",
    "sum-f-lig-I-1",
    "sum-f-lig-I-2",
    "sum-f-lig-I-3",
    "sum-f-lig-T-0",
    "sum-f-lig-T-1",
    "sum-f-lig-T-2",
    "sum-f-lig-T-3",
    "sum-f-lig-S-0",
    "sum-f-lig-S-1",
    "sum-f-lig-S-2",
    "sum-f-lig-S-3",
    "sum-lc-chi-0-all",
    "sum-lc-chi-1-all",
    "sum-lc-chi-2-all",
    "sum-lc-chi-3-all",
    "sum-lc-Z-0-all",
    "sum-lc-Z-1-all",
    "sum-lc-Z-2-all",
    "sum-lc-Z-3-all",
    "sum-lc-I-0-all",
    "sum-lc-I-1-all",
    "sum-lc-I-2-all",
    "sum-lc-I-3-all",
    "sum-lc-T-0-all",
    "sum-lc-T-1-all",
    "sum-lc-T-2-all",
    "sum-lc-T-3-all",
    "sum-lc-S-0-all",
    "sum-lc-S-1-all",
    "sum-lc-S-2-all",
    "sum-lc-S-3-all",
    "sum-lc-alpha-0-all",
    "sum-lc-alpha-1-all",
    "sum-lc-alpha-2-all",
    "sum-lc-alpha-3-all",
    "sum-D_lc-chi-0-all",
    "sum-D_lc-chi-1-all",
    "sum-D_lc-chi-2-all",
    "sum-D_lc-chi-3-all",
    "sum-D_lc-Z-0-all",
    "sum-D_lc-Z-1-all",
    "sum-D_lc-Z-2-all",
    "sum-D_lc-Z-3-all",
    "sum-D_lc-I-0-all",
    "sum-D_lc-I-1-all",
    "sum-D_lc-I-2-all",
    "sum-D_lc-I-3-all",
    "sum-D_lc-T-0-all",
    "sum-D_lc-T-1-all",
    "sum-D_lc-T-2-all",
    "sum-D_lc-T-3-all",
    "sum-D_lc-S-0-all",
    "sum-D_lc-S-1-all",
    "sum-D_lc-S-2-all",
    "sum-D_lc-S-3-all",
    "sum-D_lc-alpha-0-all",
    "sum-D_lc-alpha-1-all",
    "sum-D_lc-alpha-2-all",
    "sum-D_lc-alpha-3-all",
]

summed_metalcenter_descriptors = [
    "sum-mc_CRY-chi-0-all",
    "sum-mc_CRY-chi-1-all",
    "sum-mc_CRY-chi-2-all",
    "sum-mc_CRY-chi-3-all",
    "sum-mc_CRY-Z-0-all",
    "sum-mc_CRY-Z-1-all",
    "sum-mc_CRY-Z-2-all",
    "sum-mc_CRY-Z-3-all",
    "sum-mc_CRY-I-0-all",
    "sum-mc_CRY-I-1-all",
    "sum-mc_CRY-I-2-all",
    "sum-mc_CRY-I-3-all",
    "sum-mc_CRY-T-0-all",
    "sum-mc_CRY-T-1-all",
    "sum-mc_CRY-T-2-all",
    "sum-mc_CRY-T-3-all",
    "sum-mc_CRY-S-0-all",
    "sum-mc_CRY-S-1-all",
    "sum-mc_CRY-S-2-all",
    "sum-mc_CRY-S-3-all",
    "sum-D_mc_CRY-chi-0-all",
    "sum-D_mc_CRY-chi-1-all",
    "sum-D_mc_CRY-chi-2-all",
    "sum-D_mc_CRY-chi-3-all",
    "sum-D_mc_CRY-Z-0-all",
    "sum-D_mc_CRY-Z-1-all",
    "sum-D_mc_CRY-Z-2-all",
    "sum-D_mc_CRY-Z-3-all",
    "sum-D_mc_CRY-I-0-all",
    "sum-D_mc_CRY-I-1-all",
    "sum-D_mc_CRY-I-2-all",
    "sum-D_mc_CRY-I-3-all",
    "sum-D_mc_CRY-T-0-all",
    "sum-D_mc_CRY-T-1-all",
    "sum-D_mc_CRY-T-2-all",
    "sum-D_mc_CRY-T-3-all",
    "sum-D_mc_CRY-S-0-all",
    "sum-D_mc_CRY-S-1-all",
    "sum-D_mc_CRY-S-2-all",
    "sum-D_mc_CRY-S-3-all",
]

summed_functionalgroup_descriptors = [
    "sum-func-chi-0-all",
    "sum-func-chi-1-all",
    "sum-func-chi-2-all",
    "sum-func-chi-3-all",
    "sum-func-Z-0-all",
    "sum-func-Z-1-all",
    "sum-func-Z-2-all",
    "sum-func-Z-3-all",
    "sum-func-I-0-all",
    "sum-func-I-1-all",
    "sum-func-I-2-all",
    "sum-func-I-3-all",
    "sum-func-T-0-all",
    "sum-func-T-1-all",
    "sum-func-T-2-all",
    "sum-func-T-3-all",
    "sum-func-S-0-all",
    "sum-func-S-1-all",
    "sum-func-S-2-all",
    "sum-func-S-3-all",
    "sum-func-alpha-0-all",
    "sum-func-alpha-1-all",
    "sum-func-alpha-2-all",
    "sum-func-alpha-3-all",
    "sum-D_func-chi-0-all",
    "sum-D_func-chi-1-all",
    "sum-D_func-chi-2-all",
    "sum-D_func-chi-3-all",
    "sum-D_func-Z-0-all",
    "sum-D_func-Z-1-all",
    "sum-D_func-Z-2-all",
    "sum-D_func-Z-3-all",
    "sum-D_func-I-0-all",
    "sum-D_func-I-1-all",
    "sum-D_func-I-2-all",
    "sum-D_func-I-3-all",
    "sum-D_func-T-0-all",
    "sum-D_func-T-1-all",
    "sum-D_func-T-2-all",
    "sum-D_func-T-3-all",
    "sum-D_func-S-0-all",
    "sum-D_func-S-1-all",
    "sum-D_func-S-2-all",
    "sum-D_func-S-3-all",
    "sum-D_func-alpha-0-all",
    "sum-D_func-alpha-1-all",
    "sum-D_func-alpha-2-all",
    "sum-D_func-alpha-3-all",
]


 $\color{DarkBlue}{\textsf{Short question}}$
- We declared a global variable to fix the random seed (`RANDOM_SEED`). Why did we do this?

$\color{Green}{\textsf{Answer}}$: The purpose of the random seed is to ensure reproducibility of the process: if someone wants to replicate the model, they can use the same seed to begin from an identical starting point. Any sequence called upon `np.random` will follow the same sequence of random numbers, yielding stable results.

## Hands-on Project: Carbon-dioxide uptake in MOFs

In this exercise we will build a model that can predict the CO$_2$ uptake of metal-organic frameworks (MOFs), which are crystalline materials consisting of inorganic metal nodes linked by organic linkers.

![MOF building principle](assets/mof_building_principle.png)

There are two main **learning goals** for this exercise: 

1. Understand the typical workflow for machine learning in materials science. We will cover exploratory data analysis (EDA) and supervised learning (KRR).

2. Get familiar with some Python packages that are useful for data analysis and visualization. 

At the end of the exercise, you will produce an interactive plot like the one below, comparing the predictions of your model against CO$_2$ computed with GCMC simulations.
The histograms show the distributions of the errors on the training set (left) and on the test set (right).



<img src="assets/result.gif" alt="Parity interactive" width="700"/>

This exercise requires a basic knowledge of Python, e.g. that you can write list comprehensions, and are able to read documentation of functions provided by Python packages.
You will be asked to provide some function arguments (indicated by `#fillme` comments).

You can execute all the following code cells by pressing SHIFT and ENTER and get informations about the functions by pressing TAB when you are between the parentheses (see the notes for more tips). 

Also the [sklearn documentation](https://scikit-learn.org/stable/user_guide.html) is a great source of reference with many explanations and examples.

In pandas dataframe (df) you can select columns using their name by running `df[columnname]`. If at any point you think that the dataset is too large for your computer, you can select a subset using `df.sample()` or by making the test set larger in the train/test split (section 2). 

## 1. Import the data

In [4]:
df = pd.read_csv(DATA_FILE)

Let's take a look at the first few rows to see if everythings seems reasonable ...

In [5]:
pd.options.display.max_columns=df.shape[1]

df.head()

,ASA [m^2/cm^3],CellV [A^3],Df,Di,Dif,NASA [m^2/cm^3],POAV [cm^3/g],POAVF,PONAV [cm^3/g],PONAVF,density [g/cm^3],MOFname,total_SA_volumetric,total_SA_gravimetric,total_POV_volumetric,total_POV_gravimetric,mc_CRY-chi-0-all,mc_CRY-chi-1-all,mc_CRY-chi-2-all,mc_CRY-chi-3-all,mc_CRY-Z-0-all,mc_CRY-Z-1-all,mc_CRY-Z-2-all,mc_CRY-Z-3-all,mc_CRY-I-0-all,mc_CRY-I-1-all,mc_CRY-I-2-all,mc_CRY-I-3-all,mc_CRY-T-0-all,mc_CRY-T-1-all,mc_CRY-T-2-all,mc_CRY-T-3-all,mc_CRY-S-0-all,mc_CRY-S-1-all,mc_CRY-S-2-all,mc_CRY-S-3-all,D_mc_CRY-chi-0-all,D_mc_CRY-chi-1-all,D_mc_CRY-chi-2-all,D_mc_CRY-chi-3-all,D_mc_CRY-Z-0-all,D_mc_CRY-Z-1-all,D_mc_CRY-Z-2-all,D_mc_CRY-Z-3-all,D_mc_CRY-I-0-all,D_mc_CRY-I-1-all,D_mc_CRY-I-2-all,D_mc_CRY-I-3-all,D_mc_CRY-T-0-all,D_mc_CRY-T-1-all,D_mc_CRY-T-2-all,D_mc_CRY-T-3-all,D_mc_CRY-S-0-all,D_mc_CRY-S-1-all,D_mc_CRY-S-2-all,D_mc_CRY-S-3-all,sum-mc_CRY-chi-0-all,sum-mc_CRY-chi-1-all,sum-mc_CRY-chi-2-all,sum-mc_CRY-chi-3-all,sum-mc_CRY-Z-0-all,sum-mc_CRY-Z-1-all,sum-mc_CRY-Z-2-all,sum-mc_CRY-Z-3-all,sum-mc_CRY-I-0-all,sum-mc_CRY-I-1-all,sum-mc_CRY-I-2-all,sum-mc_CRY-I-3-all,sum-mc_CRY-T-0-all,sum-mc_CRY-T-1-all,sum-mc_CRY-T-2-all,sum-mc_CRY-T-3-all,sum-mc_CRY-S-0-all,sum-mc_CRY-S-1-all,sum-mc_CRY-S-2-all,sum-mc_CRY-S-3-all,sum-D_mc_CRY-chi-0-all,sum-D_mc_CRY-chi-1-all,sum-D_mc_CRY-chi-2-all,sum-D_mc_CRY-chi-3-all,sum-D_mc_CRY-Z-0-all,sum-D_mc_CRY-Z-1-all,sum-D_mc_CRY-Z-2-all,sum-D_mc_CRY-Z-3-all,sum-D_mc_CRY-I-0-all,sum-D_mc_CRY-I-1-all,sum-D_mc_CRY-I-2-all,sum-D_mc_CRY-I-3-all,sum-D_mc_CRY-T-0-all,sum-D_mc_CRY-T-1-all,sum-D_mc_CRY-T-2-all,sum-D_mc_CRY-T-3-all,sum-D_mc_CRY-S-0-all,sum-D_mc_CRY-S-1-all,sum-D_mc_CRY-S-2-all,sum-D_mc_CRY-S-3-all,lc-chi-0-all,lc-chi-1-all,lc-chi-2-all,lc-chi-3-all,lc-Z-0-all,lc-Z-1-all,lc-Z-2-all,lc-Z-3-all,lc-I-0-all,lc-I-1-all,lc-I-2-all,lc-I-3-all,lc-T-0-all,lc-T-1-all,lc-T-2-all,lc-T-3-all,lc-S-0-all,lc-S-1-all,lc-S-2-all,lc-S-3-all,lc-alpha-0-all,lc-alpha-1-all,lc-alpha-2-all,lc-alpha-3-all,D_lc-chi-0-all,D_lc-chi-1-all,D_lc-chi-2-all,D_lc-chi-3-all,D_lc-Z-0-all,D_lc-Z-1-all,D_lc-Z-2-all,D_lc-Z-3-all,D_lc-I-0-all,D_lc-I-1-all,D_lc-I-2-all,D_lc-I-3-all,D_lc-T-0-all,D_lc-T-1-all,D_lc-T-2-all,D_lc-T-3-all,D_lc-S-0-all,D_lc-S-1-all,D_lc-S-2-all,D_lc-S-3-all,D_lc-alpha-0-all,D_lc-alpha-1-all,D_lc-alpha-2-all,D_lc-alpha-3-all,func-chi-0-all,func-chi-1-all,func-chi-2-all,func-chi-3-all,func-Z-0-all,func-Z-1-all,func-Z-2-all,func-Z-3-all,func-I-0-all,func-I-1-all,func-I-2-all,func-I-3-all,func-T-0-all,func-T-1-all,func-T-2-all,func-T-3-all,func-S-0-all,func-S-1-all,func-S-2-all,func-S-3-all,func-alpha-0-all,func-alpha-1-all,func-alpha-2-all,func-alpha-3-all,D_func-chi-0-all,D_func-chi-1-all,D_func-chi-2-all,D_func-chi-3-all,D_func-Z-0-all,D_func-Z-1-all,D_func-Z-2-all,D_func-Z-3-all,D_func-I-0-all,D_func-I-1-all,D_func-I-2-all,D_func-I-3-all,D_func-T-0-all,D_func-T-1-all,D_func-T-2-all,D_func-T-3-all,D_func-S-0-all,D_func-S-1-all,D_func-S-2-all,D_func-S-3-all,D_func-alpha-0-all,D_func-alpha-1-all,D_func-alpha-2-all,D_func-alpha-3-all,f-lig-chi-0,f-lig-chi-1,f-lig-chi-2,f-lig-chi-3,f-lig-Z-0,f-lig-Z-1,f-lig-Z-2,f-lig-Z-3,f-lig-I-0,f-lig-I-1,f-lig-I-2,f-lig-I-3,f-lig-T-0,f-lig-T-1,f-lig-T-2,f-lig-T-3,f-lig-S-0,f-lig-S-1,f-lig-S-2,f-lig-S-3,sum-lc-chi-0-all,sum-lc-chi-1-all,sum-lc-chi-2-all,sum-lc-chi-3-all,sum-lc-Z-0-all,sum-lc-Z-1-all,sum-lc-Z-2-all,sum-lc-Z-3-all,sum-lc-I-0-all,sum-lc-I-1-all,sum-lc-I-2-all,sum-lc-I-3-all,sum-lc-T-0-all,sum-lc-T-1-all,sum-lc-T-2-all,sum-lc-T-3-all,sum-lc-S-0-all,sum-lc-S-1-all,sum-lc-S-2-all,sum-lc-S-3-all,sum-lc-alpha-0-all,sum-lc-alpha-1-all,sum-lc-alpha-2-all,sum-lc-alpha-3-all,sum-D_lc-chi-0-all,sum-D_lc-chi-1-all,sum-D_lc-chi-2-all,sum-D_lc-chi-3-all,sum-D_lc-Z-0-all,sum-D_lc-Z-1-all,sum-D_lc-Z-2-all,sum-D_lc-Z-3-all,sum-D_lc-I-0-all,sum-D_lc-I-1-all,sum-D_lc-I-2-all,sum-D_lc-I-3-all,sum-D_lc-T-0-all,sum-D_lc-T-1-all,sum-D_lc-T-2-all,sum-D_lc-T-3-all,sum-D_lc-S-0-all,sum-D_lc-S-1-all,sum-D_lc-S-2-all,sum-D_lc-S-3-all,sum-D_lc-alpha-0-all,sum-D_lc-alpha-1-a

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li>Use something like <code>pd.options.display.max_columns=100</code> to adjust how many columns are shown.<code>pd.options.display.max_columns=100</code>  would show at maximum 100 columns. </li>
</ul>
</details>

Let's also get some basic information ...

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Columns: 343 entries, ASA [m^2/cm^3] to CH4LPSTP
dtypes: float64(342), object(1)
memory usage: 45.5+ MB


 $\color{DarkBlue}{\textsf{Short question}}$
- How many materials are in the dataset?

$\color{Green}{\textsf{Answer}}$: We have 17379 materials in the data set

- Which data types do we deal with?

$\color{Green}{\textsf{Answer}}$: We deal with 342 float64 and 1 object. The object is the "MOFname" column.


Below, we define three global variables (hence upper case), which are the *names* of our feature and target columns. We will use the `TARGET` for the actual regression and the `TARGET_BINARY` only for the stratified train/test split. The `FEATURES` variable is a list of column names of our dataframe.

In [8]:
TARGET = "pure_uptake_CO2_298.00_1600000"
TARGET_BINARY = "target_binned"  # will be created later
FEATURES = (
    geometric_descriptors
    + summed_functionalgroup_descriptors
    + summed_linker_descriptors
    + summed_metalcenter_descriptors
)


As descriptors we will use geometric properties such as density, pore volume, etc. and [revised autocorrelation functions](https://pubs.acs.org/doi/abs/10.1021/acs.jpca.7b08750) (RACs) that have been optimized for describing inorganic compounds ([and recently adapated for MOFs](https://www.nature.com/articles/s41467-020-17755-8))

Examples for pore geometry descriptors (in `geometric_descriptors`) include: $D_i$ (the size of the largest included sphere), $D_f$ (the largest free sphere), and $D_{if}$ (the largest included free sphere) along the pore $-$ three ways of characterizing pore size. 

![pore diameters](assets/spheres.png)

Also included are the surface area (SA) of the pore, and the probe-occupiable pore volume (POV).
More details on the description of pore geometries can be found in [Ongari et al.](https://pubs.acs.org/doi/abs/10.1021/acs.langmuir.7b01682)

RACs (in the lists starting with `summed_...`) operate on the structure graph and encode information about the metal center, linkers and the functional groups as differences or products of heuristics that are relevant for inorganic chemistry, such as electronegativity ($\chi$), connectivity ($T$), identity ($I$), covalent radii ($S$), and nuclear charge ($Z$).


<img src="assets/racs.png" alt="RACs scheme from the lecture" width="700"/>

The number in the descriptornames shows the coordination shell that was considered in the calculation of the RACs.

The target we use for this application is the high-pressure CO$_2$ uptake. This is the amount of CO$_2$ (mmol) the MOF can load per gram.

## 2. Split the data

Next, we split our data into a training set and a test set.

In order to prevent *any* information of the test set from leaking into our model, we split *before* starting to analyze or transform our data. For more details on why this matters, see [chapter 7.10.2 of Elements of Statistical Learning](https://web.stanford.edu/~hastie/ElemStatLearn//printings/ESLII_print10.pdf).

### 2.1. Split with stratification

[Stratification](https://en.wikipedia.org/wiki/Stratified_sampling) ensures that the class distributions (ratio of "good" to "bad" materials) are the same in the training and test set.

 $\color{DarkBlue}{\textsf{Short question}}$

- Why is this important? What could happen if we would not do this? 

$\color{Green}{\textsf{Answer}}$: Stratification helps train the model with the same ratio of classes that will be found in the testing set. If stratification is not performed, it is to say, if one of the two sets, for instance, the test set, has more "good" materials; the model may appear as performing well but would fail over "bad" materials. The same could happen the other way round yielding a poor model performance because it was only trained on "bad" materials.

For stratification to work, we to define what makes a "good" or a "bad" material. We will use 15 mmol CO$_2$ / g as the threshold for the uptake, thus binarizing our continuous target variable. (You can choose it based on the histogram of the variables).

 $\color{DarkBlue}{\textsf{Short Exercise}}$
 - add a column 'target_binary' that encodes whether a material is low performing (`0`) or high perfoming (`1`) by comparing the uptake with the `THRESHOLD`

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> you can use <a href='https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html'>pd.cut</a>, 
    <a href='https://stackoverflow.com/questions/4406389/if-else-in-a-list-comprehension'>list comprehension</a>, the <a href='https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Binarizer.html#sklearn.preprocessing.Binarizer'> binarizer in sklearn </a>...) </li>
    <li> a list comprehension example: <code> [1 if value > THRESHOLD else 0 for value in df[TARGET]] </code> </li>
</ul>
</details>

In [9]:
THRESHOLD = 15 # in units of mmol CO2/g
df[TARGET_BINARY] = [1 if value > THRESHOLD else 0 for value in df[TARGET]]

Now, we can perform the actual split into training and test set.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- select reasonable values for `XX` and `XY` and then perform the test/train splits. What do you consider when making this decision (think about what you would do with really small and really big datasets, what happens if you have only one test point, what happens to the model performance if you have more test points than training points)? 
- why do we need to perform the split into a training and test set? 
- would we use the test set to tune the hyperparameters of our model?

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li>The `size` arguments can either be integers or, often more convenient, decimals like 0.1</li>
    <li>When you perform the split into training and test set you need to trade-off bias (pessimistic bias due to little training data) and variance (due to little test data) </li>
    <li>A typical split cloud be 70/30, but for huge dataset the test set might be too big and for small datasets the training set might be too small in this way </li>
</ul>
</details>

In [10]:
df_train_stratified, df_test_stratified = train_test_split(
    df,
    train_size=0.7,
    test_size=0.3,
    random_state=RANDOM_SEED,
    stratify=df[TARGET_BINARY],
)


## 3. Exploratory data analysis (EDA) 

After we have put the test set aside, we can give the training set a closer look.

### 3.1. Correlations

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Plot some features against the target property and calculate the Pearson and Spearman correlation coefficient (what is the different between those correlation coefficients?)

$\color{Green}{\textsf{Answer}}$: The Pearson and Spearman correlation coefficients both measure the strength and direction of a relationship between two variables. The **Pearson correlation coefficient** is designed to measure linear relationships and relies on the covariance of continuous, normally distributed variables. It’s sensitive to outliers, as it depends on actual data values. It produces a value of +1 for a perfect positive linear relationship, -1 for a perfect negative linear relationship, and 0 for no linear association. In contrast, the **Spearman correlation coefficient** measures the strength of monotonic relationships by using ranked values, making it a non-parametric measure suitable for ordinal data or data with outliers. A Spearman coefficient of +1 or -1 indicates a perfect monotonic relationship, while 0 signifies no monotonic association, making it more robust for datasets that do not meet normality assumptions.
**Therefore, the major difference between the two coefficients is the type of relationship they measure.**

- What are the strongest correlations? Did you expect them?

$\color{Green}{\textsf{Answer}}$: The Spearman correlation coefficients are greater in all three graphs compared to the Pearson correlation coefficient. When the Spearman correlation coefficient is greater than the Pearson correlation coefficient, it suggests a consistent but non-linear relationship between the variables. 
Listed below are the Spearman correlation coefficients of each graph. 

- Spearman Correlation:
  - density [g/cm<sup>3</sup>]: -0.948727
  - ASA [m<sup>2</sup>/cm<sup>3</sup>]: 0.859978
  - POAV [cm<sup>3</sup>/g]: 0.923432

One can see that the **density** and the **pure_uptake_CO2_298.00_1600000** have the strongest negative correlation coefficient. Specifically, this suggests that as the density of the MOF increases, the amount of carbon dioxide (CO₂) it can adsorb (measured as pure uptake at 298 K and a pressure of 1,600,000 Pa) tends to decrease.

This inverse correlation may indicate that denser MOFs have less available pore space or fewer active sites for CO₂ adsorption, resulting in lower uptake capacity. In other words, the structural characteristics that contribute to higher density might limit the MOF's effectiveness in capturing CO₂, which is an important consideration in designing materials for carbon capture applications. Understanding this relationship can help guide the development of MOFs that optimize both density and CO₂ uptake for better performance in environmental and industrial applications.

Similarly, we strongly correlate the **PAOV** and the **pure_uptake_CO2_298.00_1600000**. However, we have a positive correlation coefficient, indicating that both increase together. 

In the context of Metal-Organic Frameworks (MOFs), a higher PAOV typically means that there is more available space within the structure for gas molecules to enter and be retained. This increased availability can enhance the adsorption capacity of CO₂. Therefore, a strong correlation between PAOV and CO₂ uptake indicates that the structural features of the MOF, which allow for a larger volume of accessible space, are critical for optimizing its performance in capturing carbon dioxide.

And finally, we see a moderate positive correlation between the **ASA** and the **pure_uptake_CO2_298.00_1600000**. This relationship indicates that a larger ASA provides more active sites for the interaction and adsorption of CO₂ molecules. A greater surface area typically allows for more contact between the gas and the solid, enhancing the material's overall capacity to capture carbon dioxide. However, since the correlation is described as moderate, it implies that while there is a tendency for increased CO₂ uptake with larger surface areas, other factors may also play a significant role in influencing the adsorption process.

- What can be a problem when features are correlated?

$\color{Green}{\textsf{Answer}}$: Correlated features in a dataset can lead to several issues including multicollinearity - which makes it hard to determine how much each feature affects the outcome because they are too similar. This can also result in overfitting, where the model learns from noise in the data instead of the real patterns, making it less accurate on new data. Additionally, having correlated features can create redundancy, meaning that some features provide the same information, which can complicate the model unnecessarily. It can also make it harder to understand the model, as it becomes difficult to know which feature is really influencing the outcome. Lastly, it can slow down the training process for the model because the presence of correlated features can make it harder for optimization algorithms to work effectively.

- *Optional:* Do they change if you switch from CO$_2$ to CH$_4$ uptake as the target instead? Explain your observation.

$\color{Green}{\textsf{Answer}}$: When switching from CO$_2$ to CH$_4$ uptake as the target property, some differences in correlations emerge, though the overall trends remain consistent. For both gases, density shows a strong negative correlation with uptake, suggesting that denser MOFs perform poorly in adsorbing both CH$_4$ and CO$_2$, likely due to reduced pore space. However, the strength of the negative correlation is slightly more pronounced for CH$_4$. On the other hand, the correlations between uptake and accessible surface area (ASA), as well as probe occupiable accessible volume (POAV), are positive for both gases. Yet, CH$_4$ uptake tends to show stronger correlations with these features, particularly with POAV, indicating that the amount of accessible space plays a more significant role in CH$_4$ adsorption. CO$_2$  uptake, while still positively correlated with ASA and POAV, appears to be more influenced by density, showing a slightly weaker relationship with surface area and volume. Overall, CH$_4$ adsorption seems to be more sensitive to the available space within the MOF structure, while CO$_2$ adsorption is affected more by the material’s density.

To get the correlation matrices, you can use the `df.corr(method=)`method on your dataframe (`df`). You might want to calculate not the full correlation matrix but just the correlation of the features with the targets

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> To get the correlation with a target, you can use indexing. E.g. <code>df.corr(method='spearman')[TARGET]</code></li>
    <li> use <code>.sort_values()</code> method on the output of `df.corr()` to sort by the value of the correlation coefficient  </li>
      <li> You can use something like <code>scatter = hv.Scatter(df, 'Di', [TARGET,  'density [g/cm^3]']).opts(color='density [g/cm^3]', cmap='rainbow')</code> for plotting. Also consider the <a href="https://holoviews.org/reference/elements/matplotlib/Scatter.html"> <code>holoviews</code> documentation</a>. In case <code>holoviews</code> is too new for you, you can of course just use <code>matplotlib</code> and something like <code>plt.scatter(x,y)</code> </li>
</ul>
</details>

In [11]:
import pandas as pd
from scipy.stats import pearsonr, spearmanr
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import row

# Use this if you are running in a Jupyter Notebook
output_notebook()

# Assuming df is your DataFrame
features = ['ASA [m^2/cm^3]', 'density [g/cm^3]', 'POAV [cm^3/g]']
target = 'pure_uptake_CO2_298.00_1600000'

# Create a list to hold all plots
plots = []

# Calculate and display correlation coefficients
for feature in features:
    pearson_corr, _ = pearsonr(df[feature], df[target])
    spearman_corr, _ = spearmanr(df[feature], df[target])
    
    print(f'Feature: {feature}')
    print(f'  Pearson correlation: {pearson_corr}')
    print(f'  Spearman correlation: {spearman_corr}')
    
    # Create a Bokeh scatter plot
    source = ColumnDataSource(data=dict(x=df[feature], y=df[target]))
    p = figure(title=f'{feature} vs {target}', 
               x_axis_label=feature, 
               y_axis_label=target, 
               width=400,  # Adjust width as needed
               height=400)  # Adjust height as needed
    p.scatter('x', 'y', source=source, alpha=0.6)
    p.grid.grid_line_alpha = 0.3
    
    # Add the plot to the list
    plots.append(p)

# Show all plots in a row layout
show(row(*plots))


Loading BokehJS ...

Feature: ASA [m^2/cm^3]
  Pearson correlation: 0.8275290072484266
  Spearman correlation: 0.8599783195386285
Feature: density [g/cm^3]
  Pearson correlation: -0.8846360549246106
  Spearman correlation: -0.9487270052266709
Feature: POAV [cm^3/g]
  Pearson correlation: 0.7867490758533784
  Spearman correlation: 0.9234319927981253


## 4. Baselines

For machine learning, it is important to have some *baselines* to which one then compares the results of a model. Think of a classification model for some rare disease where we only have 1% postives. A classification model that only predictes the negatives *all the time* will still have a amazingly high accuracy. To be able to understand if our model is really better than such a simple prediction we need to make the simple prediction first. This is what we call a baseline.

A baseline could be a really simple model, a basic heuristic or the current state of the art.
this. We will use a heuristic.

For this we use sklearn `Dummy` objects that simply calculate the mean, the median or the most frequent case of the training set, when you run the `fit()` method on them (which takes the features matrix $\mathbf{X}$ and the labels $\mathbf{y}$ as arguments.
This is, the prediction of a `DummyRegressor` with `mean` strategy will always be the mean, independent of the input (it will not look at the feature matrix!). 

Instead of using those `sklearn` objects you could also just manually compute the the mean or median of the dataset. But we will use those objects as we can learn in this way how to use estimators in `sklearn` and it is also allows you to test your full pipeline with different (baseline) models. 
What does this mean? In practice this means that you can use all the regression and classification models shown in the figure below in the same way, they will all have a `fit()` method that accepts `X` and `y` and a predict method that accepts `X` and returns the predictions. 


<img src="https://scikit-learn.org/stable/assets/ml_map.png" alt="ML Map" width="800"/>

The estimator objects can be always used in the same way 

<img src="https://static.packt-cdn.com/products/9781789800265/graphics/d49a2e95-8f22-42ed-89f1-474b3d028787.png" alt="ML Map" width="400"/>

Using these objects, instead of the mean directly, allows you to easily swap them with other models in pipelines, where one chains many data transformation steps (see section 6).

### 4.1. Build dummy models

$\color{DarkBlue}{\textsf{Short Question}}$
- If you call `.fit(X, y)` on a `DummyRegressor` does it actually use the `X`? If not, why is there still the place for the `X` in the function? If yes, how does it use it?

$\color{DarkBlue}{\textsf{Short Exercise}}$
- Create [`DummyRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html) instances for  `mean`, `median`. (e.g. `dummyinstance = DummyRegressor(strategy='mean')`)
- Train them on the training data (`dummyinstance.fit(df_train[FEATURES], df_train[TARGET])`)

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> to create <code>DummyRegressor</code> you can for example use <code> dummyregressor_mean = DummyRegressor(strategy='mean') </code> </li>
    <li> to see the implementation of the <code>DummyRegressor</code> you can check out <a href="https://github.com/scikit-learn/scikit-learn/blob/73732e5a0bc9b72c7049dc699d69aaedbb70ef0a/sklearn/dummy.py#L391"> the source code on GitHub</a> </li>
</ul>
</details>

In [11]:
# Build DummyRegressors
dummyregressor_mean = DummyRegressor(strategy='mean')
dummyregressor_median = DummyRegressor(strategy='median')

In [12]:
# Fit Dummy Regressors
dummyregressor_mean.fit(df_train_stratified[FEATURES], df_train_stratified[TARGET])
dummyregressor_median.fit(df_train_stratified[FEATURES], df_train_stratified[TARGET])

DummyRegressor(strategy='median')

#### Evaluate the performance of the dummy models

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Calculate maximum error, mean absolute error and mean square error for the dummy regressors on training and test set. What would you expect those numbers to be?
- Do the actual values surprise you? 
- What does this mean in practice for reporting of metrics/the reasoning behind using baseline models

It can be handy to store our metrics of choice in a nested dictionary ([Python dictionaries are key-value pairs](https://www.tutorialspoint.com/python/python_dictionary.htm)): 

```python
{
    'dummyestimator1': {
                        'metric_a_key': metric_a_value, 
                        'metric_b_key': metric_b_value
                    },
    'dummyestimator2': {
                        'metric_a_key': metric_a_value, 
                        'metric_b_key': metric_b_value
                    },
 }
``` 

You will now write functions `get_regression_metrics(model, X, y_true)` that compute the metrics and return this dictionary for a given model. The `predict` method takes the feature matrix $\mathbf{X}$ as input.

In them, we calculate 

$\mathrm {MAE} =\frac{\sum _{i=1}^{n}\left|Y_{i}-\hat{y}_{i}\right|}{n}.$

and 

$\mathrm {MSE} = {\frac {1}{n}}\sum _{i=1}^{n}(Y_{i}-{\hat {Y_{i}}})^{2}.$ 

where $\hat{y}$ are the predictions and, $Y_{i}$ the true values.

as well as the maximum error.

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> to perform a prediction using a estimator object, you can call <code> classifier.predict(X) </code> </li>
    <li> to calculate metrics, you can for example call <code>accuracy_score(true_values, predicted_values) </code> </li>
</ul>
</details>

In [13]:
def get_regression_metrics(model, X, y_true):
    """
    Get a dicionary with regression metrics:

    model: sklearn model with predict method
    X: feature matrix
    y_true: ground truth labels
    """
    y_predicted = model.predict(X)

    mae = mean_absolute_error(y_true, y_predicted)
    mse = mean_squared_error(y_true, y_predicted)
    maximum_error = max_error(y_true, y_predicted)

    metrics_dict = {
        'mae': mae,
        'mse': mse,
        'max_error': maximum_error
    }

    return metrics_dict

In [14]:
dummy_regressors = [
    ('mean', dummyregressor_mean),
    ('median', dummyregressor_median)
]

In [15]:
dummy_regressor_results_test = {} # initialize empty dictionary
dummy_regressor_results_train = {}

# loop over the dummy_regressor list
# if you have a tuple regressorname, regressor = (a, b) that is automatically expanded into the variables
# a = regressorname, b = regressor
for regressorname, regressor in dummy_regressors:
    print(f"Calculating metrics for {regressorname}")
    dummy_regressor_results_test[regressorname] = get_regression_metrics(regressor, df_test_stratified[FEATURES], df_test_stratified[TARGET])
    dummy_regressor_results_train[regressorname] = get_regression_metrics(regressor, df_test_stratified[FEATURES], df_test_stratified[TARGET])

print("Training Metrics:", dummy_regressor_results_train)
print("Testing Metrics:", dummy_regressor_results_test)

Calculating metrics for mean
Calculating metrics for median
Training Metrics: {'mean': {'mae': 6.27366716404819, 'mse': 53.03274968687297, 'max_error': 22.680388292469132}, 'median': {'mae': 6.183402451368086, 'mse': 54.69101599465944, 'max_error': 24.056926324899997}}
Testing Metrics: {'mean': {'mae': 6.27366716404819, 'mse': 53.03274968687297, 'max_error': 22.680388292469132}, 'median': {'mae': 6.183402451368086, 'mse': 54.69101599465944, 'max_error': 24.056926324899997}}


## 5. Build actual regression models

Let's build a simple [kernel ridge regression (KRR)](https://emtiyaz.github.io/pcml15/kernel-ridge-regression.pdf) machine learning model and train it with our raw data.
You can try different kernels, but we recommend to start with the Gaussian radial basis function ('rbf') kernel.
 
 $\color{DarkBlue}{\textsf{Short Question}}$
- Do you expect this model to perform better than the dummy models?
- Train it and then calculate the performance metrics on the training and test set. How do they compare to the performance of the dummy models?
- What is the shape of the Kernel and of the weights? (you can check your answer by looking at the `dual_coef_` attribute of the KRR instance. You can get shapes of objects using the `shape` atrribute 

In [16]:
# Train the model with a Gaussian kernel
krr = KernelRidge(kernel='rbf')
krr.fit(df_test_stratified[FEATURES], df_test_stratified[TARGET])

KernelRidge(kernel='rbf')

In [17]:
# get the metrics on the train and the test set using the get_regression_metrics functions (as above)
krr_metrics_train = get_regression_metrics(krr, df_test_stratified[FEATURES], df_test_stratified[TARGET])
krr_metrics_test = get_regression_metrics(krr, df_test_stratified[FEATURES], df_test_stratified[TARGET])


print("Kernel Ridge Regression Metrics on Training Set:")
print(krr_metrics_train)

print("\nKernel Ridge Regression Metrics on Test Set:")
print(krr_metrics_test)

Kernel Ridge Regression Metrics on Training Set:
{'mae': 5.858345170282213, 'mse': 47.59655585759982, 'max_error': 17.246056071300004}

Kernel Ridge Regression Metrics on Test Set:
{'mae': 5.858345170282213, 'mse': 47.59655585759982, 'max_error': 17.246056071300004}


## 6. Evaluate the model performance in detail

We have trained our first machine learning model!
We'll first have a closer look at its performance, before learning how to improve it.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Create a parity plot (true values against predictions)for the training and test data

- Plot a histogram of the distribution of the training and test errors on the training and test set. Plot the errors also as a function of the true value

- Let's assume we would like to use our model for pre-screening a library of millions of porous materials to zoom in on those with the most promising gas uptake. Could you tolerate the errors of your model?

$\color{Green}{\textsf{Answer}}$: The error of the test set is too big, therefore it can't be tolerated. Thus, if we predict the MOF's capability to adsorb CO<sub>2</sub>
, the prediction would be skewed and our pre-screening would not work.

- Compare the parity plots for this model with the ones for the dummy models. 
Use the plotting functions below the evaluate all the following models you train.

For this exercise, it can be handy to save the results in a dictionary, e.g. 
```(python)
res_train = {
    'y true': [],
    'y pred': []
}
```
$\color{Green}{\textsf{Answer}}$: Below are the parity plots for the KRR model and the two dummy models (mean and median). One can see that the parity plots for the dummy models could not be plotted like the KKR model due to the division by zero, causing an error. However, it is still possible to draw its shape with a linear curve. Thus, by comparing the plots, one can see that all the models fit badly the data. Nevertheless, the KRR model predicts slightly the trend as the points are closer to the y = x curve, but still not ideal.




<details>
<summary> <font color='green'>Click here for hints for plotting</font></summary>
<ul>
    <li> If you want to use matplotlib to make the parity plots, you can use the <a href="https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.hist2d.html">hist2d function</a> </li>
    <li> To create the frequencies and the edges of a histogram, one can use <code>np.histogram</code></li>
</ul>
</details>

In [18]:
# Create dictionaries with training and test results to create parity plots
res_train = {
    'y true': df_train_stratified[TARGET],
    'y pred': krr.predict(df_train_stratified[FEATURES])
}

res_test = {
    'y true': df_test_stratified[TARGET],
    'y pred': krr.predict(df_test_stratified[FEATURES])
}

Now, lets calculate the errors

In [19]:
res_train["error"] = res_train["y true"] - res_train["y pred"]
res_test["error"] = res_test["y true"] - res_test["y pred"]

Now, plot the parity plots and error distributions

<details>
<summary> <font color='green'>Click here for hints for plotting</font></summary>
If you want interactive plots, you can use the following code:
<pre><code>
hv.extension("bokeh")
hex_train = hv.HexTiles(res_train, ["y true", "y pred"]).hist(
    dimension=["y true", "y pred"]
)
hex_test = hv.HexTiles(res_test, ["y true", "y pred"]).hist(
    dimension=["y true", "y pred"]
)
hex_train + hex_test
</code>
</details>

In [ ]:
# plot it
import holoviews as hv

hv.extension('bokeh')
hex_train = hv.HexTiles(res_train, ['y true', 'y pred']).hist(dimension=['y true','y pred'])
hex_test = hv.HexTiles(res_test, ['y true', 'y pred']).hist(dimension=['y true', 'y pred'])

x = np.linspace(0,35, 1000)

line_curve = hv.Curve((x, x), 'x', 'y').opts(color='red', line_dash='dashed')



hex_train * line_curve + hex_test* line_curve


In [ ]:
# Create dictionaries with training and test results to create parity plots for dummy mean model
res_train_dummy_mean = {
    'y true': df_train_stratified[TARGET],
    'y pred': dummyregressor_mean.predict(df_train_stratified[FEATURES])
}

res_test_dummy_mean = {
    'y true': df_test_stratified[TARGET],
    'y pred': dummyregressor_mean.predict(df_test_stratified[FEATURES])
}


res_train_dummy_mean['error'] = res_train_dummy_mean['y true'] - res_train_dummy_mean['y pred'] 
res_test_dummy_mean['error'] = res_test_dummy_mean['y true'] - res_test_dummy_mean['y pred']

# plot it
hv.extension('bokeh')
hex_train_dummy_mean = hv.HexTiles(res_train_dummy_mean, ['y true', 'y pred']).hist(dimension=['y true','y pred'])
hex_test_dummy_mean = hv.HexTiles(res_test_dummy_mean, ['y true', 'y pred']).hist(dimension=['y true', 'y pred'])


line_mean_train = hv.Curve((res_train_dummy_mean['y true'], res_train_dummy_mean['y pred']), 'x', 'y').opts(color='red')
line_mean_test = hv.Curve((res_test_dummy_mean['y true'], res_test_dummy_mean['y pred']), 'x', 'y').opts(color='red')

hex_train_dummy_mean * line_mean_train + hex_test_dummy_mean* line_mean_test


In [ ]:
# Create dictionaries with training and test results to create parity plots for dummy median model
res_train_dummy_median = {
    'y true': df_train_stratified[TARGET],
    'y pred': dummyregressor_median.predict(df_train_stratified[FEATURES])
}

res_test_dummy_median = {
    'y true': df_test_stratified[TARGET],
    'y pred': dummyregressor_median.predict(df_test_stratified[FEATURES])
}


res_train_dummy_median['error'] = res_train_dummy_median['y true'] - res_train_dummy_median['y pred'] 
res_test_dummy_median['error'] = res_test_dummy_median['y true'] - res_test_dummy_median['y pred']

# plot it
hv.extension('bokeh')
hex_train_dummy_median= hv.HexTiles(res_train_dummy_median, ['y true', 'y pred']).hist(dimension=['y true','y pred'])
hex_test_dummy_median = hv.HexTiles(res_test_dummy_median, ['y true', 'y pred']).hist(dimension=['y true', 'y pred'])

line_median_train = hv.Curve((res_train_dummy_median['y true'], res_train_dummy_median['y pred']), 'x', 'y').opts(color='red')
line_median_test = hv.Curve((res_test_dummy_median['y true'], res_test_dummy_median['y pred']), 'x', 'y').opts(color='red')

hex_train_dummy_median * line_median_train + hex_test_dummy_median* line_median_test


## 7. Improve the model 

Our training set still has a couple of issues you might have noticed:
- The feature values are not scaled (different features are measured in different units ...)
- Some features are basically constant, i.e. do not contain relevant information and just increase the dimensionality of the problem 
- Some feature distributions are skewed (which is more relevant for some models than for others ...)

$\color{DarkBlue}{\textsf{Short Question}}$
- Why might the scaling of the features be relevant for a machine-learning model?

$\color{Green}{\textsf{Answer}}$: 
In machine learning, scaling features is crucial for standardizing the data across each variable, making them comparable. When features are on different scales, those with larger magnitudes can disproportionately influence the model's performance, leading to biased results. By scaling the features, we ensure that each one contributes equally to the training process. This uniformity allows the model to learn more effectively, as it can accurately capture relationships between features without being skewed by their varying scales. Overall, proper scaling enhances the model's convergence speed and improves its predictive accuracy.

### 7.1. Standard scaling and building a first pipeline 

Given that we will now go beyond training a single model, we will build [Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), which are objects that can collect a selection of transformations and estimators. This makes it quite easy to apply the same set of operations to different datasets. A simple pipeline might be built as follows 

<img src="https://vitalflux.com/wp-content/uploads/2020/08/ML-Pipeline-Page-2-1024x307.png" alt="Pipeline" width="800"/>



 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Build a pipline that first performs standard scaling and then fits a KRR. Call it `pipe_w_scaling`. 
- Fit it on the training set 
- Make predictions, calculate the errors and make the parity plots

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> the <code>fit</code>, <code>predict</code> methods also work for pipelines </li>
</ul>
</details>

In [ ]:
hv.extension('bokeh')

pipe_w_scaling = Pipeline(
   [
       ('scaling', StandardScaler()),
       ('krr', KernelRidge(kernel='rbf'))
   ]
)


# Fit the pipeline on the training set
pipe_w_scaling.fit(df_train_stratified[FEATURES], df_train_stratified[TARGET])

# Calculate errors
res_train = {
    'y true': df_train_stratified[TARGET],
    'y pred': pipe_w_scaling.predict(df_train_stratified[FEATURES])
}

res_test = {
    'y true': df_test_stratified[TARGET],
    'y pred': pipe_w_scaling.predict(df_test_stratified[FEATURES])
}

res_train["error"] = res_train["y true"] - res_train["y pred"]
res_test["error"] = res_test["y true"] - res_test["y pred"]

# Create parity plots
hex_train = hv.HexTiles(res_train, ['y true', 'y pred']).hist(dimension=['y true','y pred'])
hex_test = hv.HexTiles(res_test, ['y true', 'y pred']).hist(dimension=['y true', 'y pred'])

x = np.linspace(0,35, 1000)

line_curve = hv.Curve((x, x), 'x', 'y').opts(color='red', line_dash='dashed')

hex_train * line_curve + hex_test* line_curve #show the combined plot 
    

### 7.2. Hyperparameter optimization

A key component we did not optimize so far are hyperparameters. Those are parameters of the model that we usually cannot learn from the data but have to fix before we train the model. 
Since we cannot learn those parameters it is not trivial to select them. Hence, what we typically do in practice is to create another set, a "validation set", and use it to test models trained with different hyperparameters.

The most common approach to hyperparameter optimization is to define a grid of all relevant parameters and to search over the grid for the best model performance.

$\color{DarkBlue}{\textsf{Short Exercise}}$
- Think about which parameters you could optimize in the pipeline. Note that your KRR model has two parameters you can optimize. You can also switch off some steps by setting them to `None'.
- For each parameter you need to define a resonable grid to search over.
- Recall, what k-fold cross-validation does. Run the hyperparameter optimization using 5-fold cross-validation (you can adjust the number of folds according to your computational resources/impatience. It turns out at k=10 is the [best tradeoff between variance and bias](https://arxiv.org/abs/1811.12808)). 
Tune the hyperparameters until you are statisfied (e.g., until you cannot improve the cross validated error any more)
- Why don't we use the test set for hyperparameter tuning but instead test on the validation set? 
- Evaluate the model performance by calculating the performance metrics (MAE, MSE, max error) on the training and the test set.
- *Optional:* Instead of grid search, try to use random search on the same grid (`RandomizedSearchCV`) and fix the number of evaluations (`n_iter`) to a fraction of the number of evaluations of grid search. What do you observe and conclude?

 $\color{DarkRed}{\textsf{Tips}}$
- If you want to see what is happening, set the `verbosity` argument of the `GridSearchCV` object to a higher number.
 
- If you want to speed up the optimization, you can run it in parallel by setting the `n_jobs` argument to the number of workers. If you set it to -1 it will use all available cores. *Using all cores might freeze your computer if you do not have enough memory*
 
- If the optimization is too slow, reduce the number of data points in your set, the number of folds or the grid size. Note that it can also be a feasible strategy to first use a coarser grid and the a finer grid for fine-tuning.

- For grid search, you need to define a parameter grid, which is a dictionary of the following form: 
```(python)
param_grid = {
                    'pipelinestage__parameter': np.logspace(-4,1,10),
                    'pipelinestage': [None, TransformerA(), TransformerB()]
            }
```

- After the search, you can access the best model with `.best_estimator_` and the best parameters with `.best_params_` on the GridSearchCV instance. For example `grid_krr.best_estimator_`

- If you initialize the GridSearchCV instance with `refit=True` it will automatically train the model with all training data (and not only the training folds from cross-validations)

The double underscore (dunder) notation works recursively and specifies the parameters for any pipeline stage. 
For example, `ovasvm__estimator__cls__C` would specifiy the `C` parameter of the estimator in the one-versus-rest classifier `ovasvm`. 

You can print all parameters of the pipeline using `print(sorted(pipeline.get_params().keys()))`

<div class="alert alert-block alert-warning">
Be aware that tight grids will drastically increase the number of experiments you will run! In some cases, it can be useful to perform the optimization in steps, i.e., first use a coarse grid and then refine in interesting regions. 
Alternatively, there are approached like <a href="https://www.jmlr.org/papers/volume18/16-558/16-558.pdf"> hyperband <a> that dynamically adjust the number of data points.
</div>

<details>
<summary> <font color='green'>Click here for hints about pipelines and grid search</font></summary>
<ul>
    <li> You can use the <code>np.logspace</code> function to generate a grid for values that you want to vary on a logarithmic scale </li>
    <li> There are two hyperparameters for KRR: the regularization strength <code>alpha</code> and the Gaussian width  <code>gamma</code> </li>
    <li> For the regularization strength, values between 1 and 1e-3 can be reasonable. For gamma you can use the median heuristic, gamma = 1 / median, or values between 1e-3 and 1e3</li>
</ul>
</details>

In [4]:
# Define the parameter grid and the grid search object
pipe_w_scaling_hyp = Pipeline(
   [
       ('scaling', StandardScaler()), 
       ('krr', KernelRidge(kernel='rbf'))
   ]
)

param_grid = {
            'scaling': [StandardScaler(), MinMaxScaler()],
            'krr__alpha': [0.0005, 0.00076, 0.00096, 0.001, 0.005],  # Test a wider range of alpha values
            'krr__gamma': [0.1, 0.15, 0.17, 0.2, 0.25],  # Expand gamma options
            }

grid_krr = GridSearchCV(pipe_w_scaling_hyp, param_grid=param_grid, 
                        cv=2, verbose=10, n_jobs=-1)

# optional random search
#random_krr = RandomizedSearchCV(#your pipeline, param_distributions=param_grid, n_iter=#number of evaluations,
#                        cv=#number of folds, verbose=2, n_jobs=2)

NameError: name 'Pipeline' is not defined

In [ ]:
# run the grid search by calling the fit method 
grid_krr.fit(df_train_stratified[FEATURES], df_train_stratified[TARGET])
# optional random search
# random_krr.fit(#fillme)

In [3]:
best_krr_model_scaling_hyp = grid_krr.best_estimator_

# get the performance metrics
print(get_regression_metrics(best_krr_model_scaling_hyp, df_test_stratified[FEATURES], df_test_stratified[TARGET]))
print(get_regression_metrics(best_krr_model_scaling_hyp, df_train_stratified[FEATURES], df_train_stratified[TARGET]))

print(grid_krr.best_params_)

# Create dictionaries with training and test results to create parity plots 
res_train_scaling_hyp = {
    'y true': df_train_stratified[TARGET],
    'y pred': best_krr_model_scaling_hyp.predict(df_train_stratified[FEATURES])
}

res_test_scaling_hyp = {
    'y true': df_test_stratified[TARGET],
    'y pred': best_krr_model_scaling_hyp.predict(df_test_stratified[FEATURES])
}

res_train_scaling_hyp['error'] = res_train_scaling_hyp['y true'] - res_train_scaling_hyp['y pred'] 
res_test_scaling_hyp['error'] = res_test_scaling_hyp['y true'] - res_test_scaling_hyp['y pred']

# plot it
hv.extension('bokeh')
hex_train = hv.HexTiles(res_train_scaling_hyp, ['y true', 'y pred']).hist(dimension=['y true','y pred'])
hex_test = hv.HexTiles(res_test_scaling_hyp, ['y true', 'y pred']).hist(dimension=['y true', 'y pred'])


x = np.linspace(0,35, 1000)

line_curve = hv.Curve((x, x), 'x', 'y').opts(color='red', line_dash='dashed')



hex_train * line_curve + hex_test* line_curve

NameError: name 'grid_krr' is not defined

<details>
<summary> <font color='green'>Click here for some more information about hyperparameter optimization</font></summary>
Grid search is not the most efficient way to perform hyperparamter optimization. Even <a href="http://jmlr.csail.mit.edu/papers/volume13/bergstra12a/bergstra12a.pdf">random search was shown to be more efficient</a>. Really efficient though are Bayesian optimization approaches like <a href='https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf)'>TPE</a>. This is implemented in the hyperopt library, which is also installed in your conda environment.
</details>

<details>
<summary> <font color='green'>Click here for hyperparameter optimization with hyperopt (advanded and optional outlook)</font></summary>
    
<b>Import the tools we need</b>
<code>
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, mix, rand, anneal, space_eval
from functools import partial
</code>    

<b>Define the grid</b>
<code>
param_hyperopt = {
    "krr__alpha": hp.loguniform("krr__alpha", np.log(0.001), np.log(10)),
    "krr__gamma": hp.loguniform("krr__gamma", np.log(0.001), np.log(10)),
}
</code> 

<b>Define the objective function</b>
<code>
def objective_function(params):
    pipe.set_params(
        **{
            "krr__alpha": params["krr__alpha"],
            "krr__gamma": params["krr__gamma"],
        }
    )
    score = cross_val_score(
        pipe, X_train, y_train, cv=10, scoring="neg_mean_absolute_error"
    ).mean()
    return {"loss": -score, "status": STATUS_OK} 
</code>

<b>We will use a search in which we mix random search, annealing and tpe</b>
<code>
trials = Trials()
mix_search = partial(
   mix.suggest,
   p_suggest=[(0.15, rand.suggest), (0.15, anneal.suggest), (0.7, tpe.suggest)],
)
</code>

<b>Now, we can minimize the objective function.</b>
<code>
best_param = fmin(
        objective_function,
        param_hyperopt,
        algo=mix_search,
        max_evals=MAX_EVALES,
        trials=trials,
        rstate=np.random.RandomState(RANDOM_SEED),
    )
</code>

</details>

## 8. Feature Engineering 

Finally, we would like to remove features with low variance. This can be done by setting a variance threshold.

$\color{DarkBlue}{\textsf{Short Question}}$
    
- What is the reasoning behind doing this? 
- When might it go wrong and why?

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Add a variance threshold to the pipeline (select the correct function argument)
- Use random search for hyperparameter optimization, retrain the pipeline, and calculate the performance metrics (max error, MAE, MSE) on the training and test set
- If you could improve the predictive performance, do not forget to also run the model for the Kaggle competition!

In [ ]:
# Define the pipeline
pipe_variance_threshold = Pipeline(
    # fillme with the pipeline steps
    [
        ('variance_treshold', VarianceThreshold(threshold=0.01)), 
        ('scaling', StandardScaler()), 
       ('krr', KernelRidge(kernel='rbf'))
    ]
)

In [ ]:
param_grid_variance_threshold = {
                    'scaling': [StandardScaler(),MinMaxScaler()],
                    'krr__alpha': [0.0001, 0.001, 0.01, 0.1, 1],
                    'krr__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
                    'variance_treshold__threshold':  [0, 10e-9, 10e-6, 0.001, 0.01, 0.1, 1, 10, 100]
            }

random_variance_threshold = RandomizedSearchCV(pipe_variance_threshold, param_distributions=param_grid_variance_threshold, n_iter=30,
                        cv=10, verbose=10, n_jobs=-1)

In [ ]:
# Fit the pipeline and run the evaluation
random_variance_threshold.fit(df_train_stratified[FEATURES], df_train_stratified[TARGET])

$\color{DarkBlue}{\textsf{Short Exercise (optional)}}$
- replace the variance threshold with a model-based feature selection 
`('feature_selection', SelectFromModel(LinearSVC(penalty="l1")))` or [any feature selection method that you would like to try](https://scikit-learn.org/stable/modules/feature_selection.html)

## 9. Saving the model

Now, that we spent so much time in optimizing our model, we do not want to loose it. 

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- use the [joblib library](https://scikit-learn.org/stable/modules/model_persistence.html) to save your model
- make sure you can load it again


In [ ]:
# Dump your model
joblib.dump(model, filename)

In [ ]:
# Try to load it again
model_loaded = joblib.load(filename)

## 10. Influence of Regularization

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- what happens if you set $\alpha$  to a really small or to large value? Why is this the case explain what the parameter means using the equation derived in the lectures?

 To test this, fix this value in one of your pipelines, retrain the models (re-optimizing the other hyperparameters) and rerun the performance evaluation.

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> Check the derivation for ridge regression and KRR in the notes. </li>
    <li> Also remember the loss landscapes we discussed in the lectures about LASSO. </li>
</ul>
</details>

## 11. Interpreting the model 

Now, that our model performs decently, we would like to know which features are mainly responsible for this, i.e. how the model performs its reasoning. 

One method to do so is the [permutation feature importance technique](https://christophm.github.io/interpretable-ml-book/feature-importance.html).

$\color{DarkBlue}{\textsf{Short question}}$

We use both descriptors that encode the pore geometry (density, pore diameters, surface areas) as well as some that describe the chemistry of the MOF (the RACs). 
- Would you expect the relative importance of these features to be different for the prediction of gas adsorption at high vs low gas pressure?

$\color{Green}{\textsf{Answer}}$: At high pressure, we anticipate that the geometry of the MOF will have a greater impact on CO₂ uptake. This is because, at elevated pressures, more gas molecules collide with the surface of the MOF, increasing the likelihood of them adhering to it, even if the chemical interactions are not particularly favorable. The article (provided in the hint) highlights that pore topology — features such as pore size and shape—plays a major role under these conditions, as larger adsorption sites enhance CO₂ capture by providing ample surface area for collisions. Conversely, at low pressure, there are fewer gas molecules available to attach to the surface, making favorable chemistry crucial for achieving high uptake. Here, functional groups with strong electrostatic interactions, like amino or hydroxyl groups, become more important for binding CO₂ molecules effectively. Thus, we can conclude that while both geometric and chemical features are important for gas adsorption, their relative significance shifts depending on the pressure conditions

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> <a href="https://pubs.acs.org/doi/abs/10.1021/acs.chemmater.8b02257">An article from Diego et al.</a> (10.1021/acs.chemmater.8b02257) gives some hints.</li>
</ul>
</details>

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Complete the function `_calculate_permutation_scores` (which we took from the `sklearn` package) and which is needed to calculate the permutation feature importance using the `permutation_importance` function. 

In [ ]:
def _calculate_permutation_scores(estimator, X, y, col_idx, random_state,
                                  n_repeats, scorer):
    """Calculate score when `col_idx` is permuted. Based on the sklearn implementation

    estimator: sklearn estimator object
    X: pd.Dataframe or np.array
    y: pd.Dataframe or np.array
    col_idx: int
    random_state: int
    n_repeats: int
    scorer: function that takes model, X and y_true as arguments
    """
    random_state = check_random_state(random_state)

    X_permuted = X.copy()
    scores = np.zeros(n_repeats)
    # get the indices
    shuffling_idx = np.arange(X.shape[0])
    for n_round in range(n_repeats):
        # FILL BELOW HERE
        # shuffle them (fill in what you want to shuffle)
        random_state.shuffle(#fillme)

        # Deal with dataframes
        if hasattr(X_permuted, "iloc"):
            # .iloc selects the indices from a dataframe and you give it [row, column]
            col = X_permuted.iloc[shuffling_idx, col_idx]
            col.index = X_permuted.index
            X_permuted.iloc[:, col_idx] = col

        # Deal with numpy arrays
        else:
            # FILL BELOW HERE
            # array indexing is [row, column]
            X_permuted[:, col_idx] = X_permuted[#fillme]

        # Get the scores
        feature_score = scorer(estimator, X_permuted, y)

        # record the scores in array
        scores[n_round] = feature_score

    return scores

Nothing to change in the function below, it just call the `_calculate_permutation_scores` function you just completed. 

In [1]:
def permutation_importance(
    estimator,
    X,
    y,
    scoring="neg_mean_absolute_error",
    n_repeats=5,
    n_jobs=2,
    random_state=None,
):
    """Permutation importance for feature evaluation
    estimator : object
        An estimator that has already been :term:`fitted` and is compatible
        with :term:`scorer`.
    X : ndarray or DataFrame, shape (n_samples, n_features)
        Data on which permutation importance will be computed.
    y : array-like or None, shape (n_samples, ) or (n_samples, n_classes)
        Targets for supervised or `None` for unsupervised.
    scoring : string, callable or None, default=None
        Scorer to use. It can be a single
        string (see :ref:`scoring_parameter`) or a callable (see
        :ref:`scoring`). If None, the estimator's default scorer is used.
    n_repeats : int, default=5
        Number of times to permute a feature.
    n_jobs : int or None, default=2
        The number of jobs to use for the computation.
        `None` means 1 unless in a :obj:`joblib.parallel_backend` context.
        `-1` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.
    random_state : int, RandomState instance, or None, default=None
        Pseudo-random number generator to control the permutations of each
        feature. See :term:`random_state`.
    """
    # Deal with dataframes
    if not hasattr(X, "iloc"):
        X = check_array(X, force_all_finite="allow-nan", dtype=None)

    # Precompute random seed from the random state to be used
    # to get a fresh independent RandomState instance for each
    # parallel call to _calculate_permutation_scores, irrespective of
    # the fact that variables are shared or not depending on the active
    # joblib backend (sequential, thread-based or process-based).
    random_state = check_random_state(random_state)
    random_seed = random_state.randint(np.iinfo(np.int32).max + 1)

    # Determine scorer from user options.
    scorer = check_scoring(estimator, scoring=scoring)
    # get the performance score on the unpermuted data
    baseline_score = scorer(estimator, X, y)

    # run the permuted evaluations in parallel for each column
    scores = Parallel(n_jobs=n_jobs)(
        delayed(_calculate_permutation_scores)(
            estimator, X, y, col_idx, random_seed, n_repeats, scorer
        )
        for col_idx in range(X.shape[1])
    )

    # get difference two
    importances = baseline_score - np.array(scores)

    # return the results (dictionary)
    return Bunch(
        importances_mean=np.mean(importances, axis=1),
        importances_std=np.std(importances, axis=1),
        importances=importances,
    )


 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Use your function to find the five most important features.

 $\color{Green}{\textsf{Answer}}$: The five most important features are sum-f-lig-S-2, sum-f-lig-S-0, total_POV_volumetric, total_SA_volumetric, sum-f-lig-I-0


- Which are they? Did you expect this result?

 $\color{Green}{\textsf{Answer}}$: This result was unexpected because we were focused on CO$_2$ uptake at high pressure and initially believed that the geometric properties would play a bigger role. However, the findings indicate that chemical features are more significant, as three of the top five important features are related to chemistry.

In [1]:
permutation_results = permutation_importance(best_krr_model_scaling_hyp, df_train_stratified[FEATURES], df_train_stratified[TARGET], n_jobs=-1)

NameError: name 'permutation_importance' is not defined

In [2]:
permutation_results["features"] = FEATURES
bars = hv.Bars(
    permutation_results, "features", ["importances_mean", "importances_std"]
).sort("importances_mean", reverse=True)
errors = hv.ErrorBars(
    permutation_results, "features", vdims=["importances_mean", "importances_std"]
).sort("importances_mean", reverse=True)

bars * errors


NameError: name 'FEATURES' is not defined

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> To get the top <emph>n</emph> indices of an array <code>a</code>, you can use <code>np.argsort(a)[-n:]</code></li>
    <li> Get the feature names from the <code>FEATURES</code> list </li> 
    <li> combined this might look like <code>np.array(FEATURES)[np.argsort(a)[-n:]]</code></li>
</ul>
</details>

<details>
<summary> <font color='green'>Click here for more information on model interpretation</font></summary>
The permutation feature importance technique is not a silver bullet, e.g. there are issues with correlated features.
However, it is likely <a href='https://explained.ai/rf-importance/'>a better choice than feature importance, like impurity decrease, derived from random forests</a>).
</details>

## 12. Submit your best model to Kaggle 

Join the [Kaggle competition](https://www.kaggle.com/competitions/molsim-2024-ml-challenge/host/launch-checklist) for this course!
For this you can: 
- try to continue optimizing your KRR model 
- try to use a new model ([browse the sklearn documentation](https://scikit-learn.org/) for ideas or check out [xgboost](https://xgboost.readthedocs.io/en/stable/)

The important parts for us here are: 
- that you make an attempt to improve your model, discuss this attempt, and use proper models to measure potential improvement 
- we will not grade you based on how "fancy" or model is or how well it performs but rather on whether you do something reasonable that is well motivated in your discussion
- you do not need to try both a model and continue optimizing your model. Doing one of them is, in principle, "enough"

Use then your best model to create a `submission.csv` with your predictions to join the competition and upload it to the competition site.


In [ ]:
kaggle_data = pd.read_csv('data/features.csv')
kaggle_predictions = #fillme.predict(kaggle_data[FEATURES])

In [ ]:
submission = pd.DataFrame({"id": kaggle_data["id"], "prediction": kaggle_predictions})

submission.to_csv("submission.csv", index=False)
